In [2]:
# imports
import requests
import pandas as pd
import os

In [3]:
api_key = os.getenv("fsq_api_key")

In [4]:
# Function to make an http request to foursqure for each bike station
def get_poi_fs(latitude, longitude, radius, api_key, categories):    
    url = f'https://api.foursquare.com/v3/places/search?ll={latitude},{longitude}&radius={radius}'

    # Create dictionary for headers
    headers = {"Accept": "application/json"}
    headers['Authorization'] = api_key
    reponse = requests.get(url, headers=headers)
    reponse_data = reponse.json()
    
    return reponse_data['results']

In [5]:
# #Create list with desired categories from Yelp
# dining_cat_ids = ['restaurants', 'bars', 'pubs']

In [6]:
# assuming api_key is a variable containing the API key
res = get_poi_fs(41.3874, 2.1686, 1000, api_key, categories=['restaurants', 'bars', 'pubs'])


In [7]:
# to explore the result of a single POI from the testing call
res[0]

{'fsq_id': '5c4714533ba767002caedd2b',
 'categories': [{'id': 13302,
   'name': 'Mediterranean Restaurant',
   'short_name': 'Mediterranean',
   'plural_name': 'Mediterranean Restaurants',
   'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mediterranean_',
    'suffix': '.png'}}],
 'chains': [],
 'closed_bucket': 'VeryLikelyOpen',
 'distance': 47,
 'geocodes': {'drop_off': {'latitude': 41.387677, 'longitude': 2.168101},
  'main': {'latitude': 41.387526, 'longitude': 2.167989},
  'roof': {'latitude': 41.387526, 'longitude': 2.167989}},
 'link': '/v3/places/5c4714533ba767002caedd2b',
 'location': {'address': 'Rambla Catalunya, 3',
  'admin_region': 'Cataluña',
  'country': 'ES',
  'cross_street': '',
  'formatted_address': 'Rambla Catalunya, 3, 08007 Barcelona Catalunya',
  'locality': 'Barcelona',
  'postcode': '08007',
  'region': 'Catalunya'},
 'name': 'Honest Greens Rambla Catalunya',
 'related_places': {},
 'timezone': 'Europe/Madrid'}

Put your parsed results into a DataFrame ___

In [8]:
# load the df of the bike stations containing the latitude and longitude
bike_stations_load1 = pd.read_csv('../data/bike_stations_barcelona.csv', index_col=0)

In [9]:
# Create an empty dataframe
foursquare_df = pd.DataFrame()

In [10]:
# Loop through the bike_stations df to extract the lat/long/id of each row and make a new http request
for index, row in bike_stations_load1.iterrows():
    # Make the http request using row attributes
    poi_fs = get_poi_fs(row['latitude'], row['longitude'], 1000, api_key, categories=None)
    # Turn the response into a dataframe
    new_df = pd.json_normalize(poi_fs, record_path=['categories'], meta=['fsq_id', 'name', 'distance', ['location', 'address']], meta_prefix='poi_', errors='ignore')
    # Add a column with the corresponding bike station id
    new_df['station_id'] = row['id'] 
    # Add the new dataframe to yelp_df
    foursquare_df = pd.concat([foursquare_df, new_df])


In [ ]:
# Drop unnecessary columns from dataframe
columns_to_drop_foursquare = ['short_name', 'plural_name', 'icon.prefix', 'icon.suffix']

foursquare_df = foursquare_df.drop(columns=columns_to_drop_foursquare)

# Rename columns
foursquare_df = foursquare_df.rename(columns={"id": "category_id", "name": "category_name", 'poi_location.address': 'poi_address'})

In [ ]:
# Reorder columns
foursquare_df = foursquare_df[['poi_fsq_id', 'poi_name', 'category_id', 'category_name', 'poi_distance', 'poi_address', 'station_id']]

NameError: name 'foursquare_df' is not defined

In [ ]:
print(foursquare_df.shape, '\n')
print(foursquare_df.info())

In [ ]:
foursquare_df.head()

In [ ]:
#save data to use in another notebook 
foursquare_df.to_csv('../data/bike_stations_barcelona_poi_foursquare.csv')